In [1]:
import asyncio
import websockets
import json
import random
import nest_asyncio

# Aplicar el parche para permitir reutilizar el bucle de eventos existente
nest_asyncio.apply()

# Estado inicial de la temperatura y el ventilador
current_temperature = 25  # Temperatura inicial
ventilador_status = 'apagado'  # Estado inicial del ventilador

# Función para generar cambios en la temperatura
def generate_temperature():
    global current_temperature
    if ventilador_status == 'encendido':
        # Si el ventilador está encendido, disminuye la temperatura lentamente
        current_temperature += (-random.random() * 0.35) + 0.1
    else:
        # Si el ventilador está apagado, la temperatura sube o fluctúa
        current_temperature += random.random() * 0.75 - 0.25

    # Asegurarse de que la temperatura no sea negativa
    current_temperature = max(current_temperature, 0)

    # Redondear la temperatura a dos decimales
    return round(current_temperature, 2)

# Función para manejar el estado del ventilador
def control_ventilator():
    global ventilador_status
    if current_temperature >= 30:
        ventilador_status = 'encendido'
    elif current_temperature <= 20:
        ventilador_status = 'apagado'
    return ventilador_status

# Función para manejar las conexiones WebSocket
async def handle_connection(websocket):
    print("Cliente WebSocket conectado")

    try:
        while True:
            # Generar temperatura y controlar el ventilador
            temperature = generate_temperature()
            ventilador_status = control_ventilator()

            # Crear el objeto de respuesta
            response_object = {
                "randomValue": temperature,
                "ventiladorStatus": ventilador_status,
            }

            # Enviar el objeto al cliente
            await websocket.send(json.dumps(response_object))

            # Esperar 1 segundo antes de enviar el siguiente valor
            await asyncio.sleep(1)

    except websockets.exceptions.ConnectionClosed:
        print("Cliente WebSocket desconectado")
    except Exception as e:
        print(f"Error en WebSocket: {e}")

# Función para iniciar el servidor WebSocket
async def start_server():
    # Iniciar el servidor WebSocket en localhost y puerto 4001
    server = await websockets.serve(handle_connection, "localhost", 4001)
    print('Servidor WebSocket iniciado en ws://localhost:4001')
    await server.wait_closed()

# Iniciar el servidor en el notebook
asyncio.create_task(start_server())


<Task pending name='Task-1' coro=<start_server() running at C:\Users\Felip\AppData\Local\Temp\ipykernel_4760\1648835899.py:67>>

Servidor WebSocket iniciado en ws://localhost:4001
Cliente WebSocket conectado
